In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-ee96  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 21 09:57 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ehzor324'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [ ]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
#################################
#  STEMMING AND STOP WORDS

def filter_tokens(tokens, tokens2remove=None, use_stemming=True):
  stemmer = PorterStemmer()
  if tokens2remove is not None:
    tokens = [tok for tok in tokens if tok not in tokens2remove]
  if use_stemming:
    tokens = [stemmer.stem(tok) for tok in tokens]
  return tokens
#################################
def word_count(text, id):
    #tokens = filter_tokens()
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    tokens = filter_tokens(tokens,all_stopwords, True)
    freq_dict = {word: tokens.count(word) for word in tokens if word not in all_stopwords}
    ret = [(word, (id, freq_dict[word])) for word in freq_dict.keys()]
    return ret

def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl)

def calculate_df(postings):
    return postings.mapValues(lambda posting_lst: len(posting_lst))

def calculate_tf(postings):
    term_count = postings.flatMap(lambda x: [(x[0], len(x[1]))])
    term_sum = term_count.reduceByKey(lambda x, y: x + y)
    term_total = term_sum.aggregateByKey(lambda x, y: x + y, lambda x, y: x + y)
    return term_total

def partition_postings_and_write(postings):
    return postings.map(lambda posting: (token2bucket_id(posting[0]), [(posting[0], posting[1])])).reduceByKey(lambda x, y: x + y).map(lambda posting: InvertedIndex.write_a_posting_list(posting,bucket_name))

def get_docs_length(postings):
    doc_counts = postings.flatMap(lambda x: [(x[0], count) for count in x[1]])
    doc_lengths = doc_counts.aggregateByKey(0, lambda x, y: x + y, lambda x, y: x + y)
    return doc_lengths



In [ ]:
# TITLE
docs_title_pairs = parquetFile.select("id","title")
docs_title_pairs_dict = dict(docs_title_pairs.collect())
with open("titles.json", "w") as _json:
    json.dump(docs_title_pairs_dict, _json)
title_src = "titles.json"
title_dst = f"gs://{bucket_name}/titles/{titles_src}"

In [ ]:
!gsutil cp $title_src $title_dst

In [ ]:
word_counts_title = docs_title_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings_titles = word_counts_title.groupByKey().mapValues(reduce_word_counts)
t2df_title = calculate_df(postings_titles).collectAsMap()
title_length = get_docs_length(postings_titles)
total_title = calculate_tf(postings_titles)
_ = partition_postings_and_write(postings_titles, "titles").collect()

In [ ]:
# collect all posting lists locations into one super-set
titles_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='titles'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      titles_posting_locs[k].extend(v)

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
inverted.posting_locs = titles_posting_locs
inverted.df = t2df_title
inverted.doc_length = title_length.collectAsMap()
inverted.term_total = total_title.collectAsMap()
inverted.write_index('.', 'title_index')
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/titles/{index_src}'

In [ ]:
!gsutil cp $index_src $index_dst

In [ ]:
# BODY
# time the index creation time
t_start = time()
# word counts map
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# partition posting lists and write out
_ = partition_postings_and_write(postings_filtered).collect()
index_const_time = time() - t_start

24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_84_78 !
24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_99_43 !
24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_101_94 !
24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_99_89 !
24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_89_113 !
24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_84_70 !
24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_87_117 !
24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_89_108 !
24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_181_2 !
24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_181_6 !
24/02/21 15:01:22 WARN BlockManagerMasterEndpoint: No mo

In [ ]:
# test index construction time
assert index_const_time < 2*60*120

In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='postings_gcp'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
        super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/postings_gcp/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 18.4 MiB/ 18.4 MiB]                                                
Operation completed over 1 objects/18.4 MiB.                                     


In [ ]:
!gsutil ls -lh $index_dst

 18.45 MiB  2024-02-21T18:26:20Z  gs://ehzor324/postings_gcp/index.pkl
TOTAL: 1 objects, 19342406 bytes (18.45 MiB)


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [ ]:
# Put your `generate_graph` function here
def generate_graph(pages):
    edges = pages.flatMap(lambda page: [(page[0], link[0]) for link in page[1]])
    edges = edges.distinct()
    vertices_src = pages.map(lambda page: (page[0],))
    vertices_anchr = pages.flatMap(lambda page: page[1]).map(lambda link: (link[0],))
    vertices = vertices_src.union(vertices_anchr).distinct()
    return edges, vertices

In [ ]:
t_start = time()
pages_links = parquetFile.select ("id","anchor_text").rdd
# construct the graph
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

24/02/21 14:17:48 WARN YarnAllocator: Container from a bad node: container_1708509413190_0003_01_000008 on host: cluster-ee96-w-1.c.e7zor3.internal. Exit status: 143. Diagnostics: [2024-02-21 14:17:48.102]Container killed on request. Exit code is 143
[2024-02-21 14:17:48.107]Container exited with a non-zero exit code 143. 
[2024-02-21 14:17:48.108]Killed by external signal
.
24/02/21 14:17:48 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 8 for reason Container from a bad node: container_1708509413190_0003_01_000008 on host: cluster-ee96-w-1.c.e7zor3.internal. Exit status: 143. Diagnostics: [2024-02-21 14:17:48.102]Container killed on request. Exit code is 143
[2024-02-21 14:17:48.107]Container exited with a non-zero exit code 143. 
[2024-02-21 14:17:48.108]Killed by external signal
.
24/02/21 14:17:48 ERROR YarnScheduler: Lost executor 8 on cluster-ee96-w-1.c.e7zor3.internal: Container from a bad node: container_1708509413190_0003_01_000008 on ho

+-------+------------------+
|     id|          pagerank|
+-------+------------------+
|3434750| 9917.889528786583|
|  10568| 5387.609470096127|
|  32927| 5284.298441243094|
|  30680| 5130.386005836191|
|5843419| 4959.648354746545|
|  68253| 4771.279909629092|
|  31717| 4488.233081199124|
|  11867| 4148.154882176615|
|  14533|3998.1437395714406|
| 645042| 3533.109297540146|
|  17867|3247.4607632514617|
|5042916|2993.2014451081495|
|4689264| 2983.576551154407|
|  14532| 2935.978529024248|
|  25391| 2904.764828582977|
|   5405| 2892.629843363033|
|4764461| 2835.556558193232|
|  15573|2785.0334936159047|
|   9316| 2783.207255728695|
|8569916| 2776.450966760945|
+-------+------------------+
only showing top 20 rows



In [ ]:
# test that PageRank computaion took less than 2 hour
assert pr_time < 2*60*60

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete
the reporting requirements for assignment #3.

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://ehzor324/postings_gcp
